In [49]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time

L'objectif de ce TP est de créer un modèle NN simple (MLP), qui utilise des couches "fully connected", pour faire la classification de chiffres manuscrites du dataset MNIST.

# Avant de se lancer

Quelque exercice basique pour comprendre des bases de TF (2.X)

In [50]:
# create TensorFlow variables et constantes
const = tf.constant(2.0, name="const")
b = tf.Variable(2.0, name='b')
c = tf.Variable(1.0, name='c')

In [51]:
const

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [52]:
b

<tf.Variable 'b:0' shape=() dtype=float32, numpy=2.0>

In [53]:
# now create some operations

# somme de b et c (tf.add)
d = tf.add(b, c, name='d')

# substraire const à c (tf.substract)
e = tf.subtract(c, const, name='e')

# multiplication entre d et e (tf.multiply)
a = tf.multiply(d, e, name='a')

In [54]:
print("Variable e is:", e)

Variable e is: tf.Tensor(-1.0, shape=(), dtype=float32)


In [55]:
print("Variable e is:", e.numpy())

Variable e is: -1.0


In [56]:
# cela est équivalent aux opérations suivantes
d = b + c
e = c - 2
a = d * e

In [57]:
print("Variable e is:", e.numpy())

Variable e is: -1.0


In [58]:
# autres opérations utiles

In [59]:
b = tf.Variable(np.arange(0, 10), name='b')
print(b)

<tf.Variable 'b:0' shape=(10,) dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>


In [60]:
# utiliser tf.cast pour transformer b de int à float 32
# puis sommer const
d = tf.cast(b, tf.float32) + const

In [61]:
d.numpy()

array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.], dtype=float32)

In [62]:
b[1].assign(10)
b[6:9].assign([10, 10, 10])
print(b)

<tf.Variable 'b:0' shape=(10,) dtype=int64, numpy=array([ 0, 10,  2,  3,  4,  5, 10, 10, 10,  9])>


In [63]:
f = b[2:5]
print(f)

tf.Tensor([2 3 4], shape=(3,), dtype=int64)


# NN Classification MNIST

### Dataset

Importer et préparer le dataset MNIST

In [64]:
# load the training dataset.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# # reshape the data by flattening the images
# x_train = np.reshape(x_train, (-1, 784))
# x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples from the training data for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [65]:
def get_batch(x_data, y_data, batch_size):
    idxs = np.random.randint(0, len(y_data), batch_size)
    return x_data[idxs,:,:], y_data[idxs]

### Modèle

Nous voulons construire un modèle MLP (multi layer perceptron).
Pour cela, nous avons besoin d'abord de définir les couches de perceptrons qui vont composer le modèle, c'est à dire de couches "fully connected" qui prendent en entrée un vecteur de valeurs $x$ et qui donnent en sortie une activation $a = f(W*x + b)$, où:

* $W$ est la matrice des poids
* $b$ est le vecteur de biais
* $f(.)$ est la fonction d'activation

Nous voulons créer un réseau MLP à deux couches. la première couche prend en entrée les données des veleurs numériques pour chaque pixel de l'image et est composé par 64 neurones avec activation relu. La deuxième couche prend en entrée les sorties de la première et sort les logits (ou les probabilités) d'appartenence à une classe, c'est à dire la prédiction de la chiffre contenue dans l'image en entrée.

Nous allons d'abord définir les poids W et b de la bonne taille pour les deux couches du réseau : ils sont des variables dans tensorflow, car ils changnet lors de l'entrainement.
Utiliser tf.random.normal pour initialiser les valeurs des poids de manière aléatoire suivant une distribution normale. Pour les W, utiliser une deviation standard de 0.03.

In [66]:
# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random.normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random.normal([300]), name='b1')

# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random.normal([64, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random.normal([10]), name='b2')

Maintenant que nous avons à disposition les poids pour construire notre modèle, nous pouvons écrire une fonction qui, à partir d'une ou plusieurs images en entrée calcule la prédiction en sortie.

Nous allons construire un modèle qui prend en entrée les images 28x28 et il les aplatie, c'est à dire ils nous donnes des vecteurs de taille 784. Ensuite il est composé de deux couches "fully connected", la première avec 64 néurones avec activation ReLU, et une couche finale de classification composée par un nombre de neurones égal au nombre de classes, et sans activation, pour pouvoir donner en sortie la valeur de logit pour chaque classe.

In [67]:
# create a NN model with the FCLayer

def nn_model(x_input, W1, b1, W2, b2):
    
    # flatten the input image from 28 x 28 to 784
    x_input = tf.reshape(x_input, (x_input.shape[0], -1))
    
    x = tf.add(tf.matmul(tf.cast(x_input, tf.float32), W1), b1)
    
    x = tf.nn.relu(x)
    
    logits = tf.add(tf.matmul(x, W2), b2)
    
    return logits

### Entrainement

Nous entraînons le modèle en utilisant une méthode déscente de gradients stochastique (avec moni-batches) avec une boucle d'entraînement personnalisée.

Tout d'abord, nous avons besoin d'un optimiseur et d'une loss function. Nous choisissons un optimiseur SGD et une loss "cross entropy", comme il s'agit d'un problème de classification

In [68]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD()

In [69]:
def loss_fn(logits, labels):
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,
                                                                              logits=logits))
    return cross_entropy

Nous pouvons maintenant passer à la boucle d'entrainement

In [70]:
batch_size = 32
epochs = 100

In [71]:
total_batch = int(len(y_train) / batch_size)

# boucle sur les epochs
for epoch in range(epochs):
    
    avg_loss = 0
    
    # boucle sur les batchs
    for i in range(total_batch):
        
        # extraire un minibatch des donnés
        batch_x, batch_y = get_batch(x_train, y_train, batch_size=batch_size)
        
        # creations des tensors avec les données d'entrainement
        batch_x = tf.Variable(batch_x)
        batch_y = tf.Variable(batch_y)
        # transformer les labels en one-hot-encoding
        batch_y = tf.one_hot(batch_y, 10)
        
        # calcul des gradients dans le "Gradient Tape"
        with tf.GradientTape() as tape:
            logits = nn_model(batch_x, W1, b1, W2, b2)
            loss = loss_fn(logits, batch_y)
        gradients = tape.gradient(loss, [W1, b1, W2, b2])
        
        # appliquer les gradients dans l'optimiseur "stochastic gradient descent" pour la mise à jour des poids
        optimizer.apply_gradients(zip(gradients, [W1, b1, W2, b2]))
        
        # mise à jour calcul loss
        avg_loss += loss / total_batch
    
    # évaluation du modèle sur les données de test (validation)
    test_logits = nn_model(x_test, W1, b1, W2, b2)
    max_idxs = tf.argmax(test_logits, axis=1)
    test_acc = np.sum(max_idxs.numpy() == y_test) / len(y_test)
    print(f"Epoch: {epoch + 1}, loss={avg_loss:.3f}, test set      accuracy={test_acc*100:.3f}%")

print("\nTraining complete!")

InvalidArgumentError: Matrix size-incompatible: In[0]: [32,300], In[1]: [64,10] [Op:MatMul]

## Biblio

* https://medium.com/analytics-vidhya/how-to-write-a-neural-network-in-tensorflow-from-scratch-without-using-keras-e056bb143d78

* https://adventuresinmachinelearning.com/python-tensorflow-tutorial/

* https://www.tensorflow.org/guide/keras/custom_layers_and_models

* https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch